We are finally using the SFP algorithm in order to avoid the use of a universe list with all possible elements which creates a lot of false signals. It uses all the algorithms from the previous version, in addition, it uses SFP version on the top of the previous algorithms to give the really most frequent words, instead of a lot of fake counts.Unlike Apple's case, we will kinda maintain length 5 strings and will pad chars if not full.

In [ ]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [ ]:
class Differential_Privacy_CMS_with_SFP():
    def __init__(self, width=None,width0=None, depth=None,depth0=None, epsilon=None, epsilon_0=None, max_coor=None):
        self.width = width
        self.width0 = width0
        self.depth = depth
        self.depth0 = depth0
        self.epsilon = epsilon
        self.epsilon_0 = epsilon_0
        self.seed = np.random.choice(np.arange(0,self.width),replace=False,size=self.depth) if width!=None else None
        self.seed0 = np.random.choice(np.arange(0,self.width0), replace=False,size=self.depth0) if width!=None else None
        self.max = max_coor

    def CLient_Side(self,d,hash_eps_indicator):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter.We have another parameter 
        more than the previous structure,hash_eps_set since this client algo is called from client SFP and then can be called either for
        fragment or for the main string.So the hash_eps_set is an indicator '''
        if hash_eps_indicator=='main':
            depth=self.depth
            width=self.width
            epsilon=self.epsilon
            seed = self.seed
        else:
            depth=self.depth0
            width=self.width0
            epsilon=self.epsilon_0
            seed=self.seed0
        j = np.random.randint(depth) 
        z_vect = np.zeros((1,width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % width
        v[0,index]=1
        val = np.exp(epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j)

    def Compute_Sketch_Matrix(self,D,hash_eps_indicator):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm.'''
        if hash_eps_indicator=='main':
            depth=self.depth
            width=self.width
            epsilon=self.epsilon
            seed = self.seed
        else:
            depth=self.depth0
            width=self.width0
            epsilon=self.epsilon_0
            seed=self.seed0
        val = np.exp(epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,width))
        manipulated_data_matrix = np.zeros((n,width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= depth*sum_vect  
        M = np.zeros((depth,width))
        for elt in enumerate(D):
            for l in range(width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

    def Count_Mean_Sketch(self,D_s,D,hash_eps_indicator): 
        '''D_s is the stream of data and this is a subset of universe of data'''
        Modified_datalist = []
        Sketch_Matrix = []
        freq_vect = {}
        length = len(D_s) 
        for elt in enumerate(D_s):
            Modified_datalist.append(self.CLient_Side(elt[1])) 
        Sketch_Matrix = self.Compute_Sketch_Matrix(Modified_datalist)
        for d in D:
            freq_vect[d] = self.Server_Side(Sketch_Matrix,d,length) 
        return freq_vect

    def Server_Side(self,Sketch_Matrix,d,length):
        '''It returns the estimated frequency of a data element given to it. So it has two parameters, data element and the length of the 
        data stream we are considering'''
        d=d[0]*self.max+d[1]
        n = length 
        frac1 = self.width/(self.width-1)
        frac2 = n/self.width
        row_sum = 0
        for i in range(self.depth):
            hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
            int_hex = int(hex_num[:5],16)
            index = int_hex % self.width
            row_sum = row_sum + Sketch_Matrix[i,index]
        avg_row_sum = row_sum/self.depth 
        subtraction = avg_row_sum - frac2 
        assumed_freq = frac1*subtraction 
        return assumed_freq

    def frequency_counter_on_average(path_list,epsilon=None, epsilon_0=None):
        '''This function runs the entire process multiple and take the running average of frequencies for reducing error'''
        u_list = [(x,y) for x in range(20) for y in range(80)]
        freq_counter_cum = {d:0 for d in u_list}
        if epsilon==None:

            for i in range(500):
                class_instance = Differential_Privacy_CMS(width0=100,depth0=100,epsilon_0=epsilon_0,90)
                freq_counter_new = class_instance.Count_Mean_Sketch(path_list, u_list, hash_eps_indicator='0')
                for d in freq_counter_new:
                    freq_counter_cum[d]=(i*freq_counter_cum[d]+freq_counter_new[d])/(i+1)
            return freq_counter_cum
        else:
            for i in range(500):
                class_instance = Differential_Privacy_CMS(100,100,epsilon,90)
                freq_counter_new = class_instance.Count_Mean_Sketch(path_list, u_list,hash_eps_indicator='main')
                for d in freq_counter_new:
                    freq_counter_cum[d]=(i*freq_counter_cum[d]+freq_counter_new[d])/(i+1)
            return freq_counter_cum
    def Client_SFP(d):
        '''d is a tuple in our example, so for converting it to a string, we are doing this d[0]*self.max+d[1] and then make it a string.'''
        l = np.random.randint(2)
        d =  str(d[0]*self.max+d[1])
        if len(d)<4:
            pad = 4-len(d)
            for i in range(pad):
                d+='0'
        output256 = hashlib.sha256(d.encode('utf-8')).hexdigest()#h(s)
        r = output256 + d[2*l:2*l+2]
        return (self.CLient_Side(r,'0'), self.Client_side(d,'main'),l)
 
    def Server_SFP(D):
        '''For all data elements client SFP creates a tuple of length 3 and call server SFP on all those elements.So D ois the set of all those tuples.'''
        beta_data = []
        for elt in D:
            beta_data.append(elt[1])
        M = self.Compute_Sketch_M
        alpha_list1=[]
        alpha_lsit2=[]
        for elt in D:
            if elt[2]==0:
                alpha_list1.append(elt[2])
            else:
                alpha_list2.append(elt[2])
        M1 = self.Compute_Sketch_Matrix(alpha_list1,'0')
        M2 = self.Compute_Sketch_Matrix(alpha_list2,'0')
        new_dict = {}


    def Sequence_Fragment_Puzzle(self,D_s,D):
        '''hjj''''

    